In [2]:
import time
start_time = time.time()
import cv2
import sqlite3
import numpy as np
from imgbeddings import imgbeddings
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image

# Database and file paths
db_name = 'C:\\Users\\asus\\Desktop\\Project\\Face Recognition\\faces3.db'
table_name = 'images'
column_image_data = 'image_data'
column_image_embedding = 'image_embedding'
casPath = "C:\\Users\\asus\\Desktop\\test\\haarcascade_frontalface_default.xml"

# Connect to database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()
faceCascade = cv2.CascadeClassifier(casPath)

# Initialize the embedder
embedder = imgbeddings()

def calculate_embedding(image):
    image_array = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_pil = Image.fromarray(image_array)
    embedding = embedder.to_embeddings([image_pil])[0]
    return embedding

def get_all_embeddings(cursor):
    cursor.execute(f"SELECT image_id, {column_image_embedding} FROM {table_name}")
    embeddings = cursor.fetchall()
    image_ids = [row[0] for row in embeddings]
    image_embeddings = [np.frombuffer(row[1], dtype=np.float32).reshape(1, -1) for row in embeddings]
    return image_ids, image_embeddings

# Example usage
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Cannot receive frame from webcam")
        break

    faces = faceCascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5, minSize=(150, 150), flags=cv2.CASCADE_SCALE_IMAGE)
    for (x, y, w, h) in faces:
        cv2.putText(frame, 'please click c', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 0), 2)
        roi = frame[y:y+h, x:x+w]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

    cv2.imshow('Webcam', frame)
    key = cv2.waitKey(1)
    if key == ord('c'):
        cv2.imwrite(filename='saved_img.jpg', img=roi)
        cap.release()
        img_new = cv2.imread('saved_img.jpg')
        image2 = cv2.fastNlMeansDenoising(img_new)
        cv2.waitKey(1)
        cv2.destroyAllWindows()
        break

# Retrieve and compare embeddings
image2_embedding = calculate_embedding(image2).reshape(1, -1)
image_ids, image_embeddings = get_all_embeddings(cursor)
similarities = [cosine_similarity(image2_embedding, embedding)[0][0] for embedding in image_embeddings]

# Find the most similar image
max_similarity = max(similarities)
if max_similarity > 0.9:
    max_index = similarities.index(max_similarity)
    similar_image_id = image_ids[max_index]
    cursor.execute(f"SELECT image_name FROM {table_name} WHERE image_id = ?", (similar_image_id,))
    result = cursor.fetchone()
    if result:
        image_name = result[0]
        print(f"Similar image found in the database! Name: {image_name}")

# Close the database connection
conn.close()

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.2f} seconds")


E:\ana\lib\site-packages\huggingface_hub\file_download.py:588: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


Similar image found in the database! Name: mom
Execution time: 20.90 seconds
